## The `critical` Construct

The following example includes several `critical` constructs. The example  illustrates a queuing model in which a task is dequeued and worked on. To guard  against multiple threads dequeuing the same task, the dequeuing operation must  be in a `critical` region. Because the two queues in this example are independent,  they are protected by `critical` constructs with different names,  _xaxis_   and  _yaxis_ .

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: critical.1c
* type: C
*/
int dequeue(float *a);
void work(int i, float *a);

void critical_example(float *x, float *y)
{
  int ix_next, iy_next;

  #pragma omp parallel shared(x, y) private(ix_next, iy_next)
  {
    #pragma omp critical (xaxis)
      ix_next = dequeue(x);
    work(ix_next, x);

    #pragma omp critical (yaxis)
      iy_next = dequeue(y);
    work(iy_next, y);
  }

}



In [ ]:

! name: critical.1f
! type: F-fixed
      SUBROUTINE CRITICAL_EXAMPLE(X, Y)

        REAL X(*), Y(*)
        INTEGER IX_NEXT, IY_NEXT

!$OMP PARALLEL SHARED(X, Y) PRIVATE(IX_NEXT, IY_NEXT)

!$OMP CRITICAL(XAXIS)
        CALL DEQUEUE(IX_NEXT, X)
!$OMP END CRITICAL(XAXIS)
        CALL WORK(IX_NEXT, X)

!$OMP CRITICAL(YAXIS)
        CALL DEQUEUE(IY_NEXT,Y)
!$OMP END CRITICAL(YAXIS)
        CALL WORK(IY_NEXT, Y)

!$OMP END PARALLEL

      END SUBROUTINE CRITICAL_EXAMPLE



The following example extends the previous example by adding the `hint` clause to the `critical` constructs.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: critical.1c
* type: C
*/
#include <omp.h>

int dequeue(float *a);
void work(int i, float *a);

void critical_example(float *x, float *y)
{
  int ix_next, iy_next;

  #pragma omp parallel shared(x, y) private(ix_next, iy_next)
  {
    #pragma omp critical (xaxis) hint(omp_lock_hint_contended)
      ix_next = dequeue(x);
    work(ix_next, x);

    #pragma omp critical (yaxis) hint(omp_lock_hint_contended)
      iy_next = dequeue(y);
    work(iy_next, y);
  }

}



In [ ]:

! name: critical.1f
! type: F-fixed
! version: omp_4.5
      SUBROUTINE CRITICAL_EXAMPLE(X, Y)
        USE OMP_LIB        ! or INCLUDE "omp_lib.h"

        REAL X(*), Y(*)
        INTEGER IX_NEXT, IY_NEXT

!$OMP PARALLEL SHARED(X, Y) PRIVATE(IX_NEXT, IY_NEXT)

!$OMP CRITICAL(XAXIS) HINT(OMP_LOCK_HINT_CONTENDED)
        CALL DEQUEUE(IX_NEXT, X)
!$OMP END CRITICAL(XAXIS)
        CALL WORK(IX_NEXT, X)

!$OMP CRITICAL(YAXIS) HINT(OMP_LOCK_HINT_CONTENDED)
        CALL DEQUEUE(IY_NEXT,Y)
!$OMP END CRITICAL(YAXIS)
        CALL WORK(IY_NEXT, Y)

!$OMP END PARALLEL

      END SUBROUTINE CRITICAL_EXAMPLE

